In [1]:
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    if len(G) == 0:
        return {}

    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G

    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()

    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())

    if personalization is None:
        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError('Personalization dictionary must have a value for every node. Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())

    if dangling is None:
        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary must have a value for every node. Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())

    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

     # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:
            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise NetworkXError('Pagerank: power iteration failed to converge in %d iterations.' % max_iter)

In [2]:
import networkx as nx

In [3]:
G = nx.barabasi_albert_graph(60, 41)
pr = nx.pagerank(G, 0.4)

In [4]:
print(pr)

{0: 0.028021904170395837, 1: 0.013185687231096605, 2: 0.012983207363138809, 3: 0.013379255853104883, 4: 0.013378275383244737, 5: 0.012156164727199603, 6: 0.01295631566105917, 7: 0.013185888828345677, 8: 0.012156972723585985, 9: 0.013161022713540266, 10: 0.013389435563468478, 11: 0.012969058513021941, 12: 0.012974583609320507, 13: 0.01339153934510063, 14: 0.012960079900033623, 15: 0.013367125930426795, 16: 0.013180679923208104, 17: 0.012752807815346715, 18: 0.0127663876898628, 19: 0.012593767072567864, 20: 0.013384572068285408, 21: 0.01197573532875651, 22: 0.01318112600098929, 23: 0.012969664102294443, 24: 0.013395116206873043, 25: 0.013590170701681874, 26: 0.01218222505786245, 27: 0.012363471852394113, 28: 0.012352465090185673, 29: 0.012971662568874016, 30: 0.013588101947637717, 31: 0.012394550334661851, 32: 0.012560097308081199, 33: 0.013385126211212668, 34: 0.01337392999619621, 35: 0.012981930453743898, 36: 0.01297171687690417, 37: 0.012953790980418581, 38: 0.012973307194770096, 39: 